In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Attention

In [2]:
?Attention

Init signature:
Attention(
    use_scale=False,
    score_mode='dot',
    dropout=0.0,
    seed=None,
    **kwargs,
)
Docstring:     
Dot-product attention layer, a.k.a. Luong-style attention.

Inputs are a list with 2 or 3 elements:
1. A `query` tensor of shape `(batch_size, Tq, dim)`.
2. A `value` tensor of shape `(batch_size, Tv, dim)`.
3. A optional `key` tensor of shape `(batch_size, Tv, dim)`. If none
    supplied, `value` will be used as a `key`.

The calculation follows the steps:
1. Calculate attention scores using `query` and `key` with shape
    `(batch_size, Tq, Tv)`.
2. Use scores to calculate a softmax distribution with shape
    `(batch_size, Tq, Tv)`.
3. Use the softmax distribution to create a linear combination of `value`
    with shape `(batch_size, Tq, dim)`.

Args:
    use_scale: If `True`, will create a scalar variable to scale the
        attention scores.
    dropout: Float between 0 and 1. Fraction of the units to drop for the
        attention scores. Defaults

### Cross-Attention in Keras

In [3]:
a = Attention()

In [4]:
v = tf.constant([
    [
        [1, 2],
        [3, 4],
    ]
], dtype=tf.float32)
v.shape

I0000 00:00:1728170802.137145   86957 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728170802.190515   86957 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728170802.191631   86957 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728170802.194559   86957 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

TensorShape([1, 2, 2])

In [5]:
k = tf.constant([
    [
        [1, 0],
        [0, 1],
    ]
], dtype=tf.float32)
k.shape

TensorShape([1, 2, 2])

In [6]:
q = tf.constant([
    [
        [1, 0],
        [0, 1],
        [1, 1],
        [0, 0]
    ]
], dtype=tf.float32)
q.shape

TensorShape([1, 4, 2])

In [7]:
out = a([q, v, k])
out.shape

TensorShape([1, 4, 2])

In [8]:
out[0].numpy()

array([[1.5378828, 2.5378828],
       [2.462117 , 3.462117 ],
       [2.       , 3.       ],
       [2.       , 3.       ]], dtype=float32)

### Cross-Attention in Tensorflow

In [9]:
similar_logit = [qm @ tf.transpose(km) for qm, km in zip(q, k)]
similar_weight = tf.math.softmax(similar_logit)
out = similar_weight @ v
out.numpy()

array([[[1.5378828, 2.5378828],
        [2.462117 , 3.462117 ],
        [2.       , 3.       ],
        [2.       , 3.       ]]], dtype=float32)

### Self-Attention in Keras

In [10]:
a([k, k, k]).numpy()

array([[[0.73105854, 0.26894143],
        [0.26894143, 0.73105854]]], dtype=float32)

### Self-Attention in Tensorflow

In [11]:
similar_logit = [km @ tf.transpose(km) for km, km in zip(k, k)]
similar_weight = tf.math.softmax(similar_logit)
out = similar_weight @ k
out.numpy()

array([[[0.73105854, 0.26894143],
        [0.26894143, 0.73105854]]], dtype=float32)

### Multi-head Attention

In [12]:
from tensorflow.keras.layers import MultiHeadAttention
?MultiHeadAttention

Init signature:
MultiHeadAttention(
    num_heads,
    key_dim,
    value_dim=None,
    dropout=0.0,
    use_bias=True,
    output_shape=None,
    attention_axes=None,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    seed=None,
    **kwargs,
)
Docstring:     
MultiHeadAttention layer.

This is an implementation of multi-headed attention as described in the
paper "Attention is all you Need"
[Vaswani et al., 2017](https://arxiv.org/abs/1706.03762).
If `query`, `key,` `value` are the same, then
this is self-attention. Each timestep in `query` attends to the
corresponding sequence in `key`, and returns a fixed-width vector.

This layer first projects `query`, `key` and `value`. These are
(effectively) a list of tensors of length `num_attention_heads`, where the
corresponding shapes are `(batch_size, <query dimensions>, key_di